In [35]:
# https://janakiev.com/blog/jupyter-virtual-envs/ to enable virtual_env within jupyter


In [ ]:
import sys
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import datetime,pickle
from random import shuffle
from PIL import Image
from skimage.io import imsave
from yoga_text_to_class_predict import load_text_model
from yogan.library.utility.ssim import ssim_score
from keras.preprocessing.image import img_to_array, load_img
from yogan.library.dcgan import DCGan

In [ ]:
# superset of 34 yoga classes for text-model not for GAN model
YOGA_CLASS=['agnistambhasana', 'ananda balasana', 'ardha pincha mayurasana',
       'ardha uttanasana', 'astavakrasana', 'bhekasana', 'bhujapidasana',
       'bitilasana', 'camatkarasana', 'chaturanga dandasana',
       'ganda bherundasana', 'garudasana', 'halasana', 'malasana',
       'marichyasana iii', 'marjaryasana', 'matsyasana',
       'parivrtta janu sirsasana', 'parivrtta trikonasana', 'parsva bakasana',
       'parsvottanasana', 'pasasana', 'salamba bhujangasana',
       'salamba sarvangasana', 'savasana', 'sukhasana',
       'supta baddha konasana', 'tolasana', 'urdhva mukha svanasana',
       'ustrasana', 'uttana shishosana', 'utthita parsvakonasana', 'vajrasana',
       'virabhadrasana iii']

# 5 classes final GAN  model names
gan_list=['epoch-5000-pasasana30012020_1411_48','agnistambhasana-epoch-5000-30012020_2203_30',
          'bitilasana-epoch-5000-01022020_0024_01',
          'epoch-5000-matsyasana30012020_1352_52',
         'bhujapidasana-epoch-5000-30012020_2205_26']
img_height = 64
model_dir_path = os.getcwd() + '/final_models'
ground_truth_img_dir = os.getcwd() + '/data/yoga/img/'
class_list=['pasasana','agnistambhasana','bitilasana','matsyasana','bhujapidasana',]
model_dict = dict(zip(class_list,gan_list))
out_path = os.getcwd() + '/data/outputs/'
pickle_path=os.getcwd()+"/final_models/cv_pickle.pk"

In [ ]:
with open(pickle_path,"rb") as f:
    cv=pickle.load(f)

In [ ]:
# text from  dataset on which model was trained on
dataset_text = ["From Revolved Squatting Toe Balance</a>, bring the top arm up and around the back to meet the bottom arm that will reach under and wrap around its respective knee to be met at the wrist by the top hand.  On the inhale, elongate the spine and on the exhale take the twist slightly deeper.  Keep the heart open and the gaze should be slightly over the top shoulder.",
                'From a seated position, stack both shins on top of each other until they are parallel to the front edge of the mat.',
                "From Box or Cakravākāsana, the ribcage is lifted with a gentle sway in the low back.  The tailbone lifts up into dog tilt.  The eyes are soft and the gaze is to the sky.",
                "Begin seated in Lotus (Padmāsana) or Half Lotus (Ardha Padmāsana) and lean back to your forearms.  Lift your chest to create an arch in your upper back.  Drop the crown of your head to the floor.  Keep your hands on your feet or rest them by your side.",
               "Come into Garland or Mālāsana pose.  Take your arms through your legs and place your palms flat on the floor behind your ankles.  You may need to slightly lift up your hips to get there.  Fingers face forward.  Work your upper arm as high up under your knees as possible.  Bend your elbows to create a shelf.  Hinge forward onto your palms until your feet begin to lift off.  Press firmly into the earth as you play with lifting one foot, then the other.  If available, lift both feet and cross your ankles.",
               ]
#text from external sources on web for same poses
external_text = ['Squat down, keeping the knees together. Lift the right arm up and take it over to the left side. Keep lowering the hips down. \
Try to get the armpit beyond the legs, reaching as far as possible. Try to get a bind behind the back, or use a strap if your hands are not reaching. Keep the bandhas activated.Keep opening the chest up. First of all, remember to find relaxation instead of stressing.',
      'Sit on a yoga mat with feet on the floor and knees bent.Bring your left foot under your right leg to the outside of your right hip. Lay the outer leg on the floor. Place the right leg on top of the left leg. Keep the right sole perpendicular to the ground.Cross your shins in a Sukhasana (easy pose).Exhale, bend forward at the groin. Locate your hands on the ground in front of you.On another exhalation, fold deeper into the pose.Stay in this yoga posture for 30-60 seconds.',
            
      'Start on your hands and knees in a "tabletop" position. Make sure your knees are set directly below your hips and your wrists, elbows and shoulders are in line and perpendicular to the floor. Center your head in a neutral position, eyes looking at the floor.As you inhale, lift your sitting bones and chest toward the ceiling, allowing your belly to sink toward the floor. Lift your head to look straight forward.Exhale, coming back to neutral "tabletop" position on your hands and knees. Repeat 10 to 20 times.This pose is often paired with Cat Pose on the exhale for a gentle, flowing vinyasa.'
      'Lie on your back on the floor with your knees bent, feet on the floor. Inhale, lift your pelvis slightly off the floor, and slide your hands, palms down, below your buttocks. Then rest your buttocks on the backs of your hands (and don’t lift them off your hands as you perform this pose). Be sure to tuck your forearms and elbows up close to the sides of your torso.Inhale and press your forearms and elbows firmly against the floor. Next press your scapulas into your back and, with an inhale, lift your upper torso and head away from the floor. Then release your head back onto the floor. Depending on how high you arch your back and lift your chest, either the back of your head or its crown will rest on the floor. There should be a minimal amount of weight on your head to avoid crunching your neck. You can keep your knees bent or straighten your legs out onto the floor. If you do the latter, keep your thighs active, and press out through the heels.Stay for 15 to 30 seconds, breathing smoothly. With an exhalation lower your torso and head to the floor. Draw your thighs up into your belly and squeeze.',
       'Jump forward from downward facing dog so that your feet will land to the outer sides of your hands. Stay in squat and move your hands behind the heels. Some need to lift the heels up to get the hands back. Try to get the legs a bit higher on your arms. For that, try to squeeze the shoulders a bit more under the legs. Move your hips back but keep your head and chest reaching forward. Try to maintain the balance and cross the legs: right over the left.To shift the balance down: Start to take the legs back, keeping the head reaching forward. In order not to collapse, keep the legs coming back, the head forward. Lower the head down slowly. To come up: press the hands. In order to get the head up, move the weight back, lowering the hips down. Think the movement as you were in a swing!'        
        
      ]
# text from  dataset on which model was trained on with shuffled sequence of instructions
#(shuffled sentence order) for each pose
shuffled_text = ["Keep the heart open and the gaze should be slightly over the top shoulder. On the inhale, elongate the spine and on the exhale take the twist slightly deeper.From Revolved Squatting Toe Balance</a>, bring the top arm up and around the back to meet the bottom arm that will reach under and wrap around its respective knee to be met at the wrist by the top hand.",
                'Stack both shins on top of each other until they are parallel to the front edge of the mat,from a seated position.',
                "Keep your hands on your feet or rest them by your side. The eyes are soft and the gaze is to the sky.The tailbone lifts up into dog tilt. From Box or Cakravākāsana, the ribcage is lifted with a gentle sway in the low back.",
                "Lean back to your forearms.  Drop the crown of your head to the floor. Lift your chest to create an arch in your upper back.Begin seated in Lotus (Padmāsana) or Half Lotus (Ardha Padmāsana).",
               "If available, lift both feet and cross your ankles. Take your arms through your legs and place your palms flat on the floor behind your ankles.\
               You may need to slightly lift up your hips to get there.Work your upper arm as high up under your knees as possible.\
               Press firmly into the earth as you play with lifting one foot, then the other. \
               Bend your elbows to create a shelf. \
               Hinge forward onto your palms until your feet begin to lift off. \
               Fingers face forward. Come into Garland or Mālāsana pose.",
               ]
blank_text = ['','','','','']

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

dt_string = datetime.datetime.now().strftime("%d%m%Y_%H%M_%S")

gan = DCGan()

texts_list = [dataset_text,external_text,shuffled_text,blank_text]
texts_names = ['dataset_text','external_text','shuffled_text','blank_text']
#wrong text cases can be identified from  wrong generated images in dataset_text

txt_index = 0
for texts in texts_list:
    class_index=0
    print(f'txt index={txt_index}')
    for text in texts:
        pose_name = class_list[class_index] # pose name truth
        class_index += 1
        ssim_scores=[]
        class_predicted = load_text_model(text,pickle_path,YOGA_CLASS) #pose name predicted 

        print(f'Class name predicted by text model={class_predicted}')
        if class_predicted not in class_list:
            print('class name predicted outside list of  trained models so switching to ensemble model ')
        else:
            print(f"Model chosen is {class_predicted,model_dict[class_predicted]}")
            gan.load_model(model_dir_path,model_dict[class_predicted])
            img_path = ground_truth_img_dir  +  class_predicted + '.png'

            true_img = img_to_array(load_img(img_path, target_size=(img_width, img_height)))
            true_img = (true_img.astype(np.float32) / 255) * 2 - 1
            for j in range(5):
                generated_image_single_class = gan.generate_image_from_text(text)
                plt.imsave(out_path+texts_names[txt_index]+'_generated_image_single_class_'+pose_name+'_result_'+str(j+1)+'_'+dt_string+".png",generated_image_single_class)
                ssim_scores.append([generated_image_single_class,ssim_score(true_img,generated_image_single_class)])
            print(f'Predicted with class prediction={class_predicted}')

        m=0
        for model_name in gan_list:
            m+=1
            gan.load_model(model_dir_path,model_name)
            for j in range(5):
                generated_image = gan.generate_image_from_text(text)
                plt.imsave(out_path+texts_names[txt_index]+'_generated_image_ensemble_'+pose_name+'_model_no_'+str(m)+'_result_'+str(j+1)+'_'+dt_string+".png",generated_image)
                ssim_scores.append([generated_image,ssim_score(true_img,generated_image)])
            
        predicted_image,score=max(ssim_scores, key = lambda x: x[1])
        print('Ensemble prediction')

        print(score)
        f, axarr = plt.subplots(6,5)
        k=0
        l=0
        for img,score in ssim_scores:
            axarr[k,l%5].imshow(img)
            l+=1
            if(l%5==0):
                k+=1
    txt_index +=1

In [ ]:
#view best image predicted based on ssim score
# %matplotlib inline
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# imgplot = plt.imshow(predicted_image)